In [1]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# from tensorflow.keras import datasets, layers, models
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
# from tensorflow.keras.layers import ZeroPadding2D, add, Input, Activation
# from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras import backend as K
# import tensorflow as tf
# from tensorflow import keras
# import matplotlib.pyplot as plt
import numpy as np
import wandb
from wandb.keras import WandbCallback
# tf.config.optimizer.set_jit(True)
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

In [2]:
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)


In [3]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [4]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [5]:
aug = ImageDataGenerator(width_shift_range=0.1,
                height_shift_range=0.1, horizontal_flip=True,fill_mode="nearest")

In [6]:
sweep_config = {

     'method': 'grid',

    'parameters': {
        'learning_rate': 
         {'values': [ 2e-2]},
        
        'reg': 
            {'values': [0.05 ,0.04, 0.01]},
        
        'decay': 
            {'values': [0.004,0.008,0.006]}
     
    
        }

    }
sweep_id = wandb.sweep(sweep_config, project= "ResNet_Cifar10_HyperParam_Adam") 

Create sweep with ID: vv5ax67l
Sweep URL: https://app.wandb.ai/roastedkernel/ResNet_Cifar10_HyperParam_Adam/sweeps/vv5ax67l


In [7]:
# def residual_module(data, K, stride, chanDim,
#                     reg=0.0001, bnEps=2e-5, bnMom=0.9, reduce=False):
    
#     shortcut = data
    
#     bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
#             momentum=bnMom)(data)
#     act1 = Activation("relu")(bn1)
    
#     conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
#                 kernel_regularizer=l2(reg))(act1)
    
    
#     bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                 momentum=bnMom)(conv1)
#     act2 = Activation("relu")(bn2)
#     conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
#                 padding="same", use_bias=False,
#     kernel_regularizer=l2(reg))(act2)
    
    
#     bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                 momentum=bnMom)(conv2)
#     act3 = Activation("relu")(bn3)
#     conv3 = Conv2D(K, (1, 1), use_bias=False,
#                 kernel_regularizer=l2(reg))(act3)
    
    
    
#     if reduce:
#         shortcut = Conv2D(K, (1, 1), strides=stride,
#             use_bias=False, kernel_regularizer=l2(reg))(act1)
    
#     x = add([conv3, shortcut])
    
    
    
#     return x






# def build_Resnet(width, height, depth, classes, stages, filters,
#                 reg=0.0001, bnEps=2e-5, bnMom=0.9):
    
    
#     inputShape = (height, width, depth)
#     chanDim = -1
#     inputs = Input(shape=inputShape)
#     x = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                             momentum=bnMom)(inputs)
#     x = Conv2D(filters[0], (3, 3), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)
    
    
    
#     for i in range(0, len(stages)):

#         stride = (1, 1) if i == 0 else (2, 2)
        
#         x = residual_module(x, filters[i + 1], stride,
#                 chanDim, reduce=True, bnEps=bnEps, bnMom=bnMom)
        
#         for j in range(0, stages[i] - 1):
            
#             x = residual_module(x, filters[i + 1],
#                     (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)
            
#     x = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                             momentum=bnMom)(x)
#     x = Activation("relu")(x)
#     x = AveragePooling2D((8, 8))(x) 
#     x = Flatten()(x)
#     x = Dense(classes, kernel_regularizer=l2(reg))(x)
#     x = Activation("softmax", dtype='float32')(x)
    
#     model = Model(inputs, x, name="resnet")
    
    
    
#     return model


In [8]:
def train():
    
    
    
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    from tensorflow.keras import datasets, layers, models
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.callbacks import ModelCheckpoint
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
    from tensorflow.keras.layers import ZeroPadding2D, add, Input, Activation
    from tensorflow.keras.regularizers import l2
    from sklearn.preprocessing import LabelBinarizer
    from tensorflow.keras.optimizers import SGD
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.datasets import cifar10
    from tensorflow.keras import backend as K
    import tensorflow as tf
    from tensorflow import keras
    import matplotlib.pyplot as plt
    import numpy as np
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    
    
    
    
    
    def residual_module(data, K, stride, chanDim,
                    reg=0.0001, bnEps=2e-5, bnMom=0.9, reduce=False):
    
        shortcut = data

        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                momentum=bnMom)(data)
        act1 = Activation("elu")(bn1)

        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
                    kernel_regularizer=l2(reg))(act1)


        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                    momentum=bnMom)(conv1)
        act2 = Activation("elu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
                    padding="same", use_bias=False,
        kernel_regularizer=l2(reg))(act2)


        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                    momentum=bnMom)(conv2)
        act3 = Activation("elu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False,
                    kernel_regularizer=l2(reg))(act3)



        if reduce:
            shortcut = Conv2D(K, (1, 1), strides=stride,
                use_bias=False, kernel_regularizer=l2(reg))(act1)

        x = add([conv3, shortcut])
    
    
    
        return x






    def build_Resnet(width, height, depth, classes, stages, filters,
                    reg=0.0001, bnEps=2e-5, bnMom=0.9):


        inputShape = (height, width, depth)
        chanDim = -1
        inputs = Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                                momentum=bnMom)(inputs)
        x = Conv2D(filters[0], (3, 3), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)



        for i in range(0, len(stages)):

            stride = (1, 1) if i == 0 else (2, 2)

            x = residual_module(x, filters[i + 1], stride,
                    chanDim, reduce=True, bnEps=bnEps, bnMom=bnMom)

            for j in range(0, stages[i] - 1):

                x = residual_module(x, filters[i + 1],
                        (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                                momentum=bnMom)(x)
        x = Activation("elu")(x)
        x = AveragePooling2D((8, 8))(x) 
        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation("softmax", dtype='float32')(x)

        model = Model(inputs, x, name="resnet")



        return model

    
    wandb.init()
    configs = {'learning_rate': 2e-2,'reg': 0.0009,'decay':0.001 }
    config = wandb.config
    config.epochs = 100
    
    
    
    model = build_Resnet(32, 32, 3, 10, (9, 9, 9),
                                (64, 64, 128, 256), reg=wandb.config.reg)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

    metric = "val_accuracy"

    checkpoint = ModelCheckpoint("ResNet_Cifar10.h5", monitor= metric, verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto')
    
    
    #n_epochs = 100
    batch = 128
#     s = 100 * len(trainX) // batch # number of steps in 20 epochs (batch size = 32)
#     learning_rate0 = keras.optimizers.schedules.ExponentialDecay(wandb.config.learning_rate, s
#                                                                  , wandb.config.decay)

    opt = Adam(wandb.config.learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07
               , decay=wandb.config.decay, amsgrad=True)
    #opt = opt = SGD(lr= wandb.config.learning_rate ,decay = wandb.config.decay ,  momentum= 0.9, nesterov=True)
    
    
    model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=['accuracy'])
    
    wandb_callback = WandbCallback(data_type= "image", validation_data=(testX, testY)
                                 , labels = labelNames)

    history = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch), epochs=config.epochs, 
                    validation_data=(testX, testY),steps_per_epoch=len(trainX) // batch,
                    callbacks=[es,checkpoint,wandb_callback], verbose=10)


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 1ohu91e8 with config:
	decay: 0.004
	learning_rate: 0.02
	reg: 0.05
wandb: Agent Started Run: 1ohu91e8


wandb: Wandb version 0.10.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.30990, saving model to ResNet_Cifar10.h5
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.30990 to 0.44130, saving model to ResNet_Cifar10.h5
Epoch 3/100

Epoch 00003: val_accuracy improved from 0.44130 to 0.50050, saving model to ResNet_Cifar10.h5
Epoch 4/100

Epoch 00004: val_accuracy improved from 0.50050 to 0.55750, saving model to ResNet_Cifar10.h5
Epoch 5/100


wandb: Network error resolved after 0:00:54.491182, resuming normal operation.



Epoch 00005: val_accuracy improved from 0.55750 to 0.59310, saving model to ResNet_Cifar10.h5
Epoch 6/100

Epoch 00006: val_accuracy improved from 0.59310 to 0.59620, saving model to ResNet_Cifar10.h5
Epoch 7/100

Epoch 00007: val_accuracy improved from 0.59620 to 0.66160, saving model to ResNet_Cifar10.h5
Epoch 8/100

Epoch 00008: val_accuracy did not improve from 0.66160
Epoch 9/100

Epoch 00009: val_accuracy improved from 0.66160 to 0.71140, saving model to ResNet_Cifar10.h5
Epoch 10/100

Epoch 00010: val_accuracy improved from 0.71140 to 0.76120, saving model to ResNet_Cifar10.h5
Epoch 11/100


...


Epoch 00048: val_accuracy did not improve from 0.89520
Epoch 49/100

Epoch 00049: val_accuracy did not improve from 0.89520
Epoch 50/100

Epoch 00050: val_accuracy did not improve from 0.89520
Epoch 51/100

Epoch 00051: val_accuracy improved from 0.89520 to 0.89850, saving model to ResNet_Cifar10.h5
Epoch 52/100

Epoch 00052: val_accuracy did not improve from 0.89850
Epoch 53/100

Epoch 00053: val_accuracy improved from 0.89850 to 0.90010, saving model to ResNet_Cifar10.h5
Epoch 54/100

Epoch 00054: val_accuracy did not improve from 0.90010
Epoch 55/100

Epoch 00055: val_accuracy did not improve from 0.90010
Epoch 56/100

Epoch 00056: val_accuracy did not improve from 0.90010
Epoch 57/100

Epoch 00057: val_accuracy did not improve from 0.90010
Epoch 58/100

Epoch 00058: val_accuracy did not improve from 0.90010
Epoch 59/100

Epoch 00059: val_accuracy did not improve from 0.90010
Epoch 60/100

Epoch 00060: val_accuracy improved from 0.90010 to 0.90080, saving model to ResNet_Cifar10.h5

wandb: Wandb version 0.10.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.32550, saving model to ResNet_Cifar10.h5
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.32550 to 0.42180, saving model to ResNet_Cifar10.h5
Epoch 3/100

Epoch 00003: val_accuracy improved from 0.42180 to 0.51250, saving model to ResNet_Cifar10.h5
Epoch 4/100

Epoch 00004: val_accuracy improved from 0.51250 to 0.54880, saving model to ResNet_Cifar10.h5
Epoch 5/100

Epoch 00005: val_accuracy did not improve from 0.54880
Epoch 6/100

Epoch 00006: val_accuracy improved from 0.54880 to 0.63280, saving model to ResNet_Cifar10.h5
Epoch 7/100

Epoch 00007: val_accuracy did not improve from 0.63280
Epoch 8/100

Epoch 00008: val_accuracy did not improve from 0.63280
Epoch 9/100

Epoch 00009: val_accuracy improved from 0.63280 to 0.69450, saving model to ResNet_Cifar10.h5
Epoch 10/100

Epoch 00010: val_accuracy improved from 0.69450 to 0.70260, saving 

wandb: Network error resolved after 0:00:20.370298, resuming normal operation.



Epoch 00014: val_accuracy did not improve from 0.76440
Epoch 15/100

Epoch 00015: val_accuracy improved from 0.76440 to 0.77810, saving model to ResNet_Cifar10.h5
Epoch 16/100
